# Predicting NBA Players Scores - K Nearest Neighbors (K-NN)

In [116]:
import numpy as np
from numpy.random import rand
import pandas as pd 
from pandas import Series, DataFrame
import math
import scipy

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sb

import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [117]:
%matplotlib inline
rcParams['figure.figsize'] = 5, 4
sb.set_style('whitegrid')

In [118]:
# read data
nba_data = pd.read_csv("nba_2013.csv")
nba_data

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013
5,Cole Aldrich,C,25,NYK,46,2,330,33,61,0.541,...,92,129,14,8,30,18,40,92,2013-2014,2013
6,LaMarcus Aldridge,PF,28,POR,69,69,2498,652,1423,0.458,...,599,765,178,63,68,123,147,1603,2013-2014,2013
7,Lavoy Allen,PF,24,TOT,65,2,1072,134,300,0.447,...,192,311,71,24,33,44,126,303,2013-2014,2013
8,Ray Allen,SG,38,MIA,73,9,1936,240,543,0.442,...,182,205,143,54,8,84,115,701,2013-2014,2013
9,Tony Allen,SG,32,MEM,55,28,1278,204,413,0.494,...,129,208,94,90,19,90,121,495,2013-2014,2013


In [119]:
# check for null data
nba_data.isnull().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [199]:
nba_data = nba_data.dropna(axis=0, how='any')
nba_data.isnull().sum()

player          0
pos             0
age             0
bref_team_id    0
g               0
gs              0
mp              0
fg              0
fga             0
fg.             0
x3p             0
x3pa            0
x3p.            0
x2p             0
x2pa            0
x2p.            0
efg.            0
ft              0
fta             0
ft.             0
orb             0
drb             0
trb             0
ast             0
stl             0
blk             0
tov             0
pf              0
pts             0
season          0
season_end      0
dtype: int64

In [205]:
features = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa',
                 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.',
                 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
target = ['pts']
x_train, x_test, y_train, y_test = train_test_split(nba_data[features], nba_data[target], test_size=0.33, random_state=17)

knnObj = neighbors.KNeighborsClassifier()
knnObj.fit(x_train, y_train)
accuracy = knnObj.score(x_test, y_test)
print(accuracy * 100)

1.257861635220126


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [206]:
predictions = knnObj.predict(x_test)

In [207]:
actual = y_test

# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)

mse

ValueError: Shape of passed values is (159, 159), indices imply (1, 159)

In [131]:
# fill null values
for col in null_columns:
    nba_data[col].fillna(nba_data[col].mean(), inplace=True)

In [135]:
float_cols = ['fg.', 'x3p.', 'x2p.','efg.', 'ft.' ]
for col in float_cols:
    nba_data[col] = nba_data[col].astype(int)
nba_data.dtypes

player          object
pos             object
age              int64
bref_team_id    object
g                int64
gs               int64
mp               int64
fg               int64
fga              int64
fg.              int64
x3p              int64
x3pa             int64
x3p.             int64
x2p              int64
x2pa             int64
x2p.             int64
efg.             int64
ft               int64
fta              int64
ft.              int64
orb              int64
drb              int64
trb              int64
ast              int64
stl              int64
blk              int64
tov              int64
pf               int64
pts              int64
season          object
season_end       int64
dtype: object

In [201]:
features = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p.', 'x2p.','efg.', 'ft.' , 'x3p', 'x3pa', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']  
# Select only the numeric columns from the NBA dataset
nba_numeric = nba_data[features]

# Normalize all of the numeric columns
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()



0.012578616352201259


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


### Splitting Data into Test and Training Datasets

In [193]:
# The columns that we will be making predictions with.
features = ['age', 'g', 'gs', 'mp', 'fg', 'fga' , 'x3p', 'x3pa', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']  
# Select only the numeric columns from the NBA dataset
X = features
# The column that we want to predict.
y = ["pts"]

In [194]:
# Scale variables using preprocessing
# X = preprocessing.scale(x)

In [195]:
x_train, x_test, y_train, y_test = train_test_split(nba_data[features], nba_data[y], test_size=0.33, random_state=17)

### Building and Training Your Model with Training Data

In [196]:
knnObj = neighbors.KNeighborsClassifier()
knnObj.fit(x_train, y_train)
accuracy = knnObj.score(x_test, y_test)
print(accuracy)

0.012578616352201259


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


### Evaluating The Model

In [185]:
y_expect = y_test
y_predict = knnObj.predict(x_test)

report = metrics.classification_report(y_expect, y_predict)
print(report)

             precision    recall  f1-score   support

          0       0.17      1.00      0.29         1
          1       0.00      0.00      0.00         1
          2       0.00      0.00      0.00         1
          3       0.00      0.00      0.00         1
          4       0.00      0.00      0.00         1
          5       0.00      0.00      0.00         1
          6       0.17      1.00      0.29         1
          9       0.00      0.00      0.00         0
         11       0.00      0.00      0.00         1
         12       0.00      0.00      0.00         1
         14       0.00      0.00      0.00         0
         15       0.00      0.00      0.00         0
         19       0.00      0.00      0.00         1
         20       0.00      0.00      0.00         1
         25       0.00      0.00      0.00         0
         26       0.00      0.00      0.00         1
         27       0.00      0.00      0.00         1
         28       0.00      0.00      0.00   

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [186]:

import random
from numpy.random import permutation

# Randomly shuffle the index of nba.
random_indices = permutation(nba_data.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(nba_data)/3)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba_data.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
train = nba_data.loc[random_indices[test_cutoff:]]

In [187]:
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']

# The column that we want to predict.
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])

In [188]:
predictions

array([[5.8080e+02],
       [1.0510e+03],
       [9.7360e+02],
       [1.4130e+03],
       [3.1080e+02],
       [1.1006e+03],
       [5.4760e+02],
       [5.1460e+02],
       [6.0360e+02],
       [6.2680e+02],
       [9.8000e+00],
       [5.8340e+02],
       [4.2400e+01],
       [1.5880e+02],
       [1.2962e+03],
       [3.1780e+02],
       [4.2740e+02],
       [4.5700e+02],
       [1.9960e+02],
       [6.6680e+02],
       [2.1020e+02],
       [4.0860e+02],
       [9.9600e+02],
       [2.9400e+01],
       [1.7500e+02],
       [6.1260e+02],
       [2.8940e+02],
       [3.7200e+01],
       [1.7500e+02],
       [8.7200e+01],
       [3.4080e+02],
       [7.9460e+02],
       [1.3728e+03],
       [2.3200e+01],
       [2.8120e+02],
       [8.3780e+02],
       [6.9660e+02],
       [1.4880e+02],
       [7.3240e+02],
       [6.8600e+01],
       [1.0276e+03],
       [1.9000e+01],
       [2.4200e+02],
       [1.2986e+03],
       [1.0760e+03],
       [4.4000e+00],
       [7.4440e+02],
       [1.470

In [189]:
    
# Get the actual values for the test set.
actual = test[y_column]

# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)

mse

pts    3462.251321
dtype: float64

In [95]:
nba_data[nba_data["player"] == "LeBron James"].iloc[0]

player          LeBron James
pos                       PF
age                       29
bref_team_id             MIA
g                         77
gs                        77
mp                      2902
fg                       767
fga                     1353
fg.                    0.567
x3p                      116
x3pa                     306
x3p.                0.379085
x2p                      651
x2pa                    1047
x2p.                0.621777
efg.                    0.61
ft                       439
fta                      585
ft.                     0.75
orb                       81
drb                      452
trb                      533
ast                      488
stl                      121
blk                       26
tov                      270
pf                       126
pts                     2089
season             2013-2014
season_end              2013
Name: 225, dtype: object